In [1]:
import time
from selenium import webdriver 
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
#gathers list of links based on entering an input into the Youtube search bar
#This uses Google Chrome
def SearchVideoUrls(search):
    #maximizing number of links with one search
    options = webdriver.ChromeOptions() #deprecation warning - fix not known
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    
    driver = webdriver.Chrome(chrome_options=options, executable_path= CHROME_DRIVER_FILEPATH)
    driver.get("https://www.youtube.com/results?search_query=")
    WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#search"))).send_keys(search)
    driver.find_element_by_css_selector("button.style-scope.ytd-searchbox#search-icon-legacy").click()
    links = [my_href.get_attribute("href") for my_href in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-video-renderer#video-title")))]
    #only need the youtube ID to scrape, so this first part of the url is removed
    links = [i.strip('https://www.youtube.com/watch?v=') for i in links]
    return links

In [8]:
CHROME_DRIVER_FILEPATH = r'c:\\chromedriver_win32\\chromedriver.exe'
keywords = ['vaccine bbc', 'vaccine cnn', 'covid-19 news', 'coronavirus news', 'vaccine msnbc', 'vaccine gma', 'vaccine misinformation', 'biden vaccine', 'moderna vaccine', 'pfizer vaccine', 'j&j vaccine', 'fauci vaccine']
video_ids = ['ZlT5fRMG4mU']
for SearchString in keywords:
    video_ids.extend(SearchVideoUrls(SearchString))
df = pd.DataFrame(video_ids, columns = ["Video_ID"])
df = df.drop_duplicates()
df.head()

C:\Users\Baile\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


,Video_ID
0,ZlT5fRMG4mU
1,ZusjGkucX0E
2,ktlatjB_M
3,WAly3INavM
4,QHltQlwTL3E


In [9]:
path= r'C:/Users/Baile/OneDrive/Documents/College/SeniorYear/Spring_2021/CMDA4864/'
df.to_csv(path + "youtube_scraping_urls_5121.csv")

In [ ]:
path= r'C:/Users/Baile/OneDrive/Documents/College/SeniorYear/Spring_2021/CMDA4864/'

df = pd.read_csv(path + "youtube_scraping_urls_32921.csv")
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.rename({'Video_ID':'Video ID'}, axis = 1)
df.head()

## Data Cleaning

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import pandas as pd 
import time
#nltk.download('stopwords')

In [40]:
path= r'C:/Users/Baile/OneDrive/Documents/College/SeniorYear/Spring_2021/CMDA4864/'
#Engine Parameter solves a UnicodeDecodeError for UTF-8
raw_comments = pd.read_csv(path + 'youtubecomments_5121.csv', engine = 'python')

In [41]:
def cleanBody(text):
    #removes extra spaces
    text = re.sub("\s+"," ", text)
    #removes punctuation
    text = re.sub("[^-9A-Za-z ]", "" , text)
    text = re.sub("-", " " , text)
    text = re.sub("vaccines", "vaccine" , text)
    #changes all to lowercase
    text = re.sub('\d+', '', text.lower())
    return text

<>:3: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \d
<ipython-input-41-8098131f77e8>:3: DeprecationWarning: invalid escape sequence \s
  text = re.sub("\s+"," ", text)
<ipython-input-41-8098131f77e8>:8: DeprecationWarning: invalid escape sequence \d
  text = re.sub('\d+', '', text.lower())


In [42]:
raw_comments['textOriginal'] = raw_comments['textOriginal'].astype(str)
comments = raw_comments.groupby('videoId')['textOriginal'].apply(' '.join).reset_index()

In [48]:
cleaned_comments = []
comments['textOriginal'] = comments['textOriginal'].apply(cleanBody)
stop = stopwords.words('english')
comments['textCleaned'] = comments['textOriginal'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
cleaned_comments = comments['textCleaned']

In [44]:
cleaned_comments


0     sounds like hilter forced chambers deads yeste...
1     anti-maskers anti-vaxxers say im fully vaccina...
2     wtf know true agenda probably placebo effectiv...
3     authorized u approved parent vaccinate childch...
4     shit better redbull lots energy feeling great ...
5     commence comments racist maga supporters vacci...
6     since election fraud covered media democrats o...
7     survival rate parent would allow experimental ...
8     good ty ufufd poor kid ty wont able children g...
9     teamtrumpcam u jonathanpekar ucsd companyversu...
10    talk sore losers asked trump supporter would g...
11    genetically modified adreno virus virus partic...
12    keeps look data im sure meant say look manipul...
13    demi-god took dumb see lied ill cured last yea...
14    wish spoke biden idea mumbling time times disl...
15    benefits virus experimental gene therapy vacci...
16    new cases people got shot adameve malefemale g...
17    bots smh weak humans let government things

## Data Analysis

In [45]:
import gensim
import gensim.corpora as corpora
from pprint import pprint
import pandas as pd
import pickle 
import os
import pyLDAvis
import pyLDAvis.gensim_models
import datetime

In [49]:
# Create Corpus
cleaned_comments = pd.Series(cleaned_comments)

for i in range(0,len(cleaned_comments)):
    cleaned_comments[i] = cleaned_comments[i].split()
    
# Create Dictionary
id2word = corpora.Dictionary(cleaned_comments)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in cleaned_comments]

In [53]:
# number of topics
num_topics = 4

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keywords in the 5 topics
pprint(lda_model.print_topics())

[(0,
  '0.020*"vaccine" + 0.013*"people" + 0.007*"get" + 0.007*"dont" + '
  '0.006*"take" + 0.006*"covid" + 0.006*"like" + 0.005*"one" + 0.005*"know" + '
  '0.005*"trump"'),
 (1,
  '0.016*"vaccine" + 0.012*"people" + 0.008*"get" + 0.007*"dont" + '
  '0.005*"covid" + 0.005*"like" + 0.005*"fauci" + 0.005*"know" + 0.005*"one" + '
  '0.004*"virus"'),
 (2,
  '0.021*"vaccine" + 0.011*"dont" + 0.010*"people" + 0.009*"get" + '
  '0.007*"covid" + 0.006*"like" + 0.005*"virus" + 0.004*"one" + 0.004*"know" + '
  '0.004*"us"'),
 (3,
  '0.014*"people" + 0.014*"vaccine" + 0.007*"dont" + 0.007*"get" + '
  '0.006*"covid" + 0.005*"like" + 0.005*"take" + 0.005*"trump" + 0.004*"would" '
  '+ 0.004*"know"')]


In [54]:
# Visualize the topics
filepath = r'C:/Users/Baile/OneDrive/Documents/College/SeniorYear/Spring_2021/CMDA4864/'
now = datetime.datetime.now()
date_time = now.strftime("%b_%d")
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join(filepath + 'ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    print('entering preparation: this can take awhile with data')
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    print('beginning pickle pickle: this can take awhile with alot of data')
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

entering preparation
beginning pickle pickle


In [55]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, filepath +'.ldavis_prepared_'+date_time+'_'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.002270 -0.003162       1        1  40.003321
2      0.003756 -0.001449       2        1  33.335810
3     -0.002533  0.001833       3        1  17.161488
1      0.001047  0.002778       4        1   9.499381, topic_info=            Term          Freq         Total Category  logprob  loglift
934       people   9744.000000   9744.000000  Default  30.0000  30.0000
1375     vaccine  14951.000000  14951.000000  Default  29.0000  29.0000
516          get   6145.000000   6145.000000  Default  28.0000  28.0000
363         dont   6546.000000   6546.000000  Default  27.0000  27.0000
1498       fauci   2784.000000   2784.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
778         make    174.593409   1772.478703   Topic4  -6.0613   0.0363
788         many    174.373650   1890.065298   Topic4  -6.0626  -0.0292
1285       think    176.845333   2190.715590   Topic4  -6.0485  -0.1628
860         need    174.068868   1915.644527   Topic4  -6.0643  -0.0444
1372  vaccinated    174.668999   2127.213736   Topic4  -6.0609  -0.1457

[397 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.473268        -
0         2  0.280071        -
0         3  0.142197        -
0         4  0.104594        -
24        1  0.507508  adverse
...     ...       ...      ...
1450      4  0.072193    years
1461      1  0.456258    youre
1461      2  0.271461    youre
1461      3  0.184797    youre
1461      4  0.087301    youre

[985 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2])